<a href="https://colab.research.google.com/github/Anirudh-R-1201/Shot_xG/blob/main/datascraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import requests
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd

In [45]:
laliga = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/11/26.json' #  1.json, 4.json, 25.json as well
#wc = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/43/3.json'

def get_json(url):
    r = requests.get(url)
    j = r.json()
    df = json_normalize(j, sep = "_")
    return df

laliga_df = get_json(laliga)
#wc_df = get_json(wc)

#matches = laliga_df.append(wc_df)
match_list = laliga_df['match_id'].tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


In [6]:
master = pd.DataFrame()

In [46]:
for match in match_list:
    temp = get_json('https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/%s.json' % match)
    temp['match_id'] = match
    master = master.append(temp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


In [47]:
master.head

<bound method NDFrame.head of                                         id  ...  goalkeeper_success_in_play
0     62fcd4a2-94ed-42f0-9c4e-d994ae217440  ...                         NaN
1     dcd47e2f-6a76-499e-abee-413417bd067e  ...                         NaN
2     1d6e44c2-fb77-479e-bb98-851952e9b3f9  ...                         NaN
3     d8eaa47d-d089-45a6-b3b4-a59ab8f65a6a  ...                         NaN
4     547584c5-1b05-46ce-b61a-cd124d20b385  ...                         NaN
...                                    ...  ...                         ...
3724  a8fc706b-1905-496d-a9cb-67e425977901  ...                         NaN
3725  49155e1c-1358-4079-99c1-4a125788acd1  ...                         NaN
3726  d1d19eab-d37c-4016-b76f-f3248e91b543  ...                         NaN
3727  b30e7b8d-ca6c-4153-a17e-012256fc295f  ...                         NaN
3728  40eb1c49-d13d-41eb-9e16-af0dfc1b6798  ...                         NaN

[534552 rows x 145 columns]>

In [48]:
master.to_csv('master.csv')
#from google.colab import files
#files.download('master.csv')

KeyboardInterrupt: ignored

In [49]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df.tail().transpose())

display_all(master)

,3724,3725,3726,3727,3728
id,a8fc706b-1905-496d-a9cb-67e425977901,49155e1c-1358-4079-99c1-4a125788acd1,d1d19eab-d37c-4016-b76f-f3248e91b543,b30e7b8d-ca6c-4153-a17e-012256fc295f,40eb1c49-d13d-41eb-9e16-af0dfc1b6798
index,3725,3726,3727,3728,3729
period,2,2,2,2,2
timestamp,00:48:03.702,00:48:38.703,00:48:40.164,00:48:43.182,00:48:43.182
minute,93,93,93,93,93
second,3,38,40,43,43
possession,184,185,185,185,185
duration,0,1.4607,0,0,0
type_id,24,30,23,34,34
type_name,Bad Behaviour,Pass,Goal Keeper,Half End,Half End


In [50]:
shots = master[master['type_id'] == 16]
passes = master[master['type_id'] == 30]

In [51]:
chains = pd.DataFrame()
for item in list(shots['possession'].unique()):
    temp = master[master['possession'] == item]
    chains = chains.append(temp)
    
chains = chains[chains['type_id'] != 17]
chains = chains[chains['type_id'] != 42]
chains = chains.reset_index().drop('level_0', axis=1)

chains['previous_event'] = chains['type_name'].shift()
chains['previous_event'] = chains['previous_event'].str.lower().str.replace(' ', '_').str.replace('*', '')
chains['previous_event'] = np.where(chains['previous_event'] == 'goal_keeper', 'goalkeeper', chains['previous_event'])

In [52]:
pre_shot_events = list(chains[chains['type_id'] == 16]['previous_event'].unique())
print(pre_shot_events)

['carry', 'pass', 'miscontrol', 'duel', 'ball_recovery', 'block', 'clearance', 'player_on', 'dribble', 'shot', 'error', 'foul_won', 'goalkeeper', 'substitution', 'shield', 'camera_off', 'injury_stoppage', 'foul_committed', 'dispossessed', 'interception', 'offside', 'tactical_shift', 'half_end', 'half_start', 'camera_on']


In [53]:
important_pre_shot = ['pass', 'ball_recovery', 'duel', 'interception', 'clearance', 'goalkeeper', 'miscontrol',
                      'error', 'block', 'dispossessed', '50/50', 'dribble']

for event in important_pre_shot:
    chains['follows_%s' % event] = np.where(chains['previous_event'] == event, 1, 0)

In [54]:
chains_filt = chains[['match_id', 'possession', 'duration']]
chains_filt.columns = ['match_id', 'possession', 'chain_duration']
chains_dur = chains_filt.pivot_table(index=['match_id', 'possession'], values='chain_duration', aggfunc='sum')
chains_dur.reset_index(inplace=True)

In [55]:
shots = chains[chains['type_id'] == 16]
shots = shots.merge(chains_dur, on=['match_id', 'possession'])

In [56]:
assisted_shots = shots[shots['shot_key_pass_id'].notna()]
assist_passes = passes[passes['pass_assisted_shot_id'].notna()]
assist_passes['pass_body_part_name'] = np.where((assist_passes['pass_body_part_name'] == 'Right Foot')
                                 | (assist_passes['pass_body_part_name'] == 'Left Foot'), 'foot',
                                np.where(assist_passes['pass_body_part_name'] == 'Head', 'head', 'other'))
assist_passes['pass_player_name'] = assist_passes['player_name']
pass_column = [i for i in assist_passes.columns if i.startswith('pass')]
assist_passes = assist_passes[pass_column]
pass_column.remove('pass_player_name')
assisted_shots = assisted_shots.drop(pass_column, axis=1)
shot_pass = assisted_shots.merge(assist_passes, left_on='id', right_on='pass_assisted_shot_id', how='left')
other_shots = shots[shots['shot_key_pass_id'].isna()]
shots = other_shots.append(shot_pass)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
shots['location'] = shots['location'].astype(str).str.strip('[]')
shots['x_loc'] = shots['location'].str.split(', ').str.get(0).str.strip().astype(float)
shots['y_loc'] = shots['location'].str.split(', ').str.get(1).str.strip().astype(float)

In [58]:
shots['goal'] = np.where(shots['shot_outcome_name'] == 'Goal', 1, 0)
shots = shots.reset_index().drop('level_0', axis=1)

shots['assisted'] = np.where(shots['shot_key_pass_id'].isna(), 0, 1)

shots['x_dist'] = 120 - shots['x_loc']
shots['y_dist'] = abs(40 - shots['y_loc'])
shots['distance'] = np.sqrt((shots['x_dist']**2 + shots['y_dist']**2))
shots['angle'] = np.arctan((shots['y_dist'] / shots['x_dist']))

shots['shot_body_part_name'] = np.where((shots['shot_body_part_name'] == 'Right Foot')
                                 | (shots['shot_body_part_name'] == 'Left Foot'), 'foot',
                                np.where(shots['shot_body_part_name'] == 'Head', 'head', 'other'))

shots['build_up_duration'] = shots['chain_duration'] - shots['duration']

shots['direct_free_kick'] = np.where((shots['play_pattern_name'] == 'From Free Kick')
                                      & (shots['pass_assisted_shot_id'].isna()), 1, 0)

shots['pass_angle'] = np.where(shots['pass_angle'] < 0, shots['pass_angle'] * -1, shots['pass_angle'])

shots = pd.concat([shots, pd.get_dummies(shots['shot_technique_name'], prefix='shot_technique')], axis=1)

shots.drop(['shot_technique_name', 'shot_technique_Backheel', 'shot_technique_Diving Header',
                   'shot_technique_Overhead Kick', 'shot_technique_Volley'], axis=1, inplace=True)

shots = shots.rename(columns={'shot_technique_Half Volley': 'shot_technique_half_volley',
                                            'shot_technique_Lob': 'shot_technique_lob',
                                            'shot_technique_Normal': 'shot_technique_normal'})

pens = shots[shots['shot_type_name'] == 'Penalty']
pens['xg'] = np.sum(pens['goal']/len(pens))
pens['xa'] = 0
pens = pens[['duration', 'distance', 'build_up_duration', 'shot_technique_half_volley', 'shot_technique_lob', 
             'shot_technique_normal', 'shot_open_goal', 'pass_height_name',
             'pass_cut_back', 'pass_through_ball', 'follows_goalkeeper', 'follows_dribble', 'pass_cross', 'goal',
             'player_name', 'team_name', 'match_id', 'x_loc', 'y_loc', 'pass_player_name', 'xg', 'xa', 'id']]
pens.set_index('id', inplace=True)
pens.to_csv('pens.csv')
shots = shots[shots['shot_type_name'] != 'Penalty']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
from itertools import chain
from ast import literal_eval

freeze_frame = shots[['shot_freeze_frame', 'id', 'x_loc', 'y_loc']]
freeze_frame_df = pd.DataFrame(freeze_frame)
#freeze_frame_df['shot_freeze_frame'] = freeze_frame_df['shot_freeze_frame'].apply(literal_eval)

def add_shot_id(row):
    return [item.update({'shot_id': row['id'], 'shot_x_loc': row['x_loc'], 'shot_y_loc': row['y_loc']}) 
            for item in row['shot_freeze_frame']]

freeze_frame_df.apply(add_shot_id, axis=1)
freeze_frame_df.head()

,shot_freeze_frame,id,x_loc,y_loc
0,"[{'location': [110.4, 27.3], 'player': {'id': ...",29cec991-ba6a-47d6-b793-045461abdaf7,111.2,26.5
1,"[{'location': [87.6, 31.3], 'player': {'id': 5...",4a9dc002-2667-47b1-b667-8c45d2b0f529,90.0,29.1
2,"[{'location': [108.0, 21.9], 'player': {'id': ...",55a668d4-355d-4fa2-b5c8-6efc0f1065c3,96.5,44.7
3,"[{'location': [115.0, 38.2], 'player': {'id': ...",66af161f-af2a-4879-9302-5f8d81c1df3b,112.2,36.4
4,"[{'location': [95.2, 58.0], 'player': {'id': 2...",78e3d880-f3c2-4682-9a97-4cd5ce5daeab,97.9,60.3


In [60]:
chained = chain.from_iterable(freeze_frame_df['shot_freeze_frame'])
ff_expanded = pd.DataFrame(list(chained))
ff_expanded.head()

,location,player,position,teammate,shot_id,shot_x_loc,shot_y_loc
0,"[110.4, 27.3]","{'id': 5492, 'name': 'Samuel Yves Umtiti'}","{'id': 5, 'name': 'Left Center Back'}",False,29cec991-ba6a-47d6-b793-045461abdaf7,111.2,26.5
1,"[95.0, 52.5]","{'id': 5211, 'name': 'Jordi Alba Ramos'}","{'id': 6, 'name': 'Left Back'}",False,29cec991-ba6a-47d6-b793-045461abdaf7,111.2,26.5
2,"[80.6, 24.8]","{'id': 5203, 'name': 'Sergio Busquets i Burgos'}","{'id': 15, 'name': 'Left Center Midfield'}",False,29cec991-ba6a-47d6-b793-045461abdaf7,111.2,26.5
3,"[84.8, 17.8]","{'id': 5470, 'name': 'Ivan Rakitić'}","{'id': 13, 'name': 'Right Center Midfield'}",False,29cec991-ba6a-47d6-b793-045461abdaf7,111.2,26.5
4,"[84.8, 4.4]","{'id': 6374, 'name': 'Nélson Cabral Semedo'}","{'id': 2, 'name': 'Right Back'}",False,29cec991-ba6a-47d6-b793-045461abdaf7,111.2,26.5


In [61]:
ff_expanded['location'] = ff_expanded['location'].astype(str).str.strip('[]')
ff_expanded['x_loc'] = ff_expanded['location'].str.split(', ').str.get(0).str.strip().astype(float)
ff_expanded['y_loc'] = ff_expanded['location'].str.split(', ').str.get(1).str.strip().astype(float)

In [62]:
ff_expanded['player_x_dist'] = abs(ff_expanded['shot_x_loc'] - ff_expanded['x_loc'])
ff_expanded['player_y_dist'] = abs(ff_expanded['shot_y_loc'] - ff_expanded['y_loc'])
ff_expanded['player_distance_from_shooter'] = np.where(ff_expanded['player_x_dist'] < 0,
                                                    -1*np.sqrt((ff_expanded['player_x_dist']**2 + ff_expanded['player_y_dist']**2)),
                                                           np.sqrt((ff_expanded['player_x_dist']**2 + ff_expanded['player_y_dist']**2)))
ff_expanded['player_angle_to_shooter'] = np.arctan((ff_expanded['player_y_dist'] / ff_expanded['player_x_dist']))

In [63]:
def area(x1, y1, x2, y2, x3, y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2.0)

def get_shot_cone(row):
    return area(row['shot_x_loc'], row['shot_x_loc'], 120, 44, 120, 36)

ff_expanded['shot_cone'] = ff_expanded.apply(get_shot_cone, axis=1)

def in_shot_cone(x1, y1, x2, y2, x3, y3, x, y): 
    A = area (x1, y1, x2, y2, x3, y3) 
    A1 = area (x, y, x2, y2, x3, y3)  
    A2 = area (x1, y1, x, y, x3, y3)   
    A3 = area (x1, y1, x2, y2, x, y) 
    if(A == A1 + A2 + A3): 
        return 1
    else: 
        return 0

def player_in_cone(row):
    return in_shot_cone(row['shot_x_loc'], row['shot_x_loc'], 120, 44, 120, 36, row['x_loc'], row['y_loc'])

ff_expanded['player_in_shot_cone'] = ff_expanded.apply(player_in_cone, axis=1)

ff_expanded['player_in_shot_cone'] = np.where(ff_expanded['player_angle_to_shooter'].isna(), 
                                              1, ff_expanded['player_in_shot_cone'])

ff_expanded['player_angle_to_shooter'] = ff_expanded['player_angle_to_shooter'].fillna(0)

In [64]:
shot_cone = pd.pivot_table(ff_expanded, index=['shot_id'], values=['shot_cone']).reset_index()
shot_cone.columns = ['id', 'shot_cone']
players_in_cone = pd.pivot_table(ff_expanded, index=['shot_id'], values=['player_in_shot_cone'], aggfunc=np.sum).reset_index()
players_in_cone.columns = ['id', 'num_players_in_shot_cone']
min_distance = pd.pivot_table(ff_expanded[ff_expanded['teammate'] == True], index=['shot_id'],
                                          values=['player_distance_from_shooter'], aggfunc=np.min).reset_index()
min_distance.columns = ['id', 'distance_to_nearest_opponent']
shots = shots.merge(shot_cone, on='id', how='left')
shots = shots.merge(players_in_cone, on='id', how='left')
shots = shots.merge(min_distance, on='id', how='left')

In [65]:
shots.set_index('id', inplace=True)

feature_cols = ['duration', 'distance', 'build_up_duration', 'shot_technique_normal', 'shot_technique_lob', 
                'shot_technique_half_volley', 'shot_open_goal', 'pass_height_name',
                'pass_cut_back', 'pass_through_ball', 'follows_goalkeeper', 'follows_dribble', 'pass_cross', 'goal',
                'player_name', 'team_name', 'match_id', 'x_loc', 'y_loc', 'pass_player_name']

shots_final = shots[feature_cols]

In [66]:
print(shots_final.isnull().sum())

duration                         0
distance                         0
build_up_duration                0
shot_technique_normal            0
shot_technique_lob               0
shot_technique_half_volley       0
shot_open_goal                3516
pass_height_name               955
pass_cut_back                 3494
pass_through_ball             3329
follows_goalkeeper               0
follows_dribble                  0
pass_cross                    3031
goal                             0
player_name                      0
team_name                        0
match_id                         0
x_loc                            0
y_loc                            0
pass_player_name               955
dtype: int64


In [67]:
bool_cols = ['pass_cross', 'pass_cut_back', 'pass_through_ball', 'shot_open_goal']

for col in bool_cols:
    shots_final[col] = np.where(shots_final[col].isna(), 0, 1)

shots_final['pass_height_name'] = np.where(shots_final['pass_height_name'] == 'Ground Pass', 0,
                                np.where(shots_final['pass_height_name'] == 'High Pass', 1,
                                np.where(shots_final['pass_height_name'] == 'Low Pass', 2, -1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
shots_final.to_csv('non_pen.csv')
files.download('pens.csv')
files.download('non_pen.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
display_all(pens)

id,2554dd62-1369-4ffa-b679-3a5d430b10eb,87ae483a-faf5-481e-a099-c399ce594313,80cf5232-3eee-4fb2-86a8-ac457fe57463,0594d7ad-1ace-48ff-a8ad-7153fc870377,acfefa82-043e-4c1d-bb6f-de1dd835c6f8
duration,0.6836,0.437,0.6495,0.5297,0.485
distance,12.0004,11.9004,12,11.9038,11.9017
build_up_duration,0,0,0,0,1.11362
shot_technique_half_volley,0,0,0,0,0
shot_technique_lob,0,0,0,0,0
shot_technique_normal,1,1,1,1,1
shot_open_goal,NaN,NaN,NaN,NaN,NaN
pass_height_name,NaN,NaN,NaN,NaN,NaN
pass_cut_back,NaN,NaN,NaN,NaN,NaN
pass_through_ball,NaN,NaN,NaN,NaN,NaN


In [70]:
display_all(shots_final)

id,5eb760d8-cdec-47a2-a7e3-a698c280985e,d96a3dd9-693b-4ced-a35e-42f39bd495f2,533a6541-1444-4625-aa19-8aabcd8fae69,9472840b-f732-4f7f-a0bc-1cd4ebd87e91,73422110-d568-4334-a558-e0c0ff0069d9
duration,0.468251,0.254,0.845132,0.402948,0.2396
distance,16.2936,28.5016,16.6481,20.7077,8.98721
build_up_duration,35.5344,38.6378,25.0244,36.3207,16.576
shot_technique_normal,1,1,1,1,1
shot_technique_lob,0,0,0,0,0
shot_technique_half_volley,0,0,0,0,0
shot_open_goal,0,0,0,0,0
pass_height_name,2,2,0,0,1
pass_cut_back,0,0,0,0,0
pass_through_ball,0,0,0,0,1
